In [129]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [130]:
words = open('names.txt','r').read().split()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [131]:
len(words)

32033

In [132]:
# building the vocabulary
stoi = {}
chars = sorted(list(set(''.join(words))))
stoi = {char:i+1 for i,char in enumerate(chars)}
stoi['.'] = 0
itos = {i:char for char,i in stoi.items()}
itos

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [175]:
block_size = 3
X,Y=[],[]

for word in words:
    
    context = [0]*block_size
    for ch in word + '.':
        ix=stoi[ch]
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

In [176]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [201]:
g = torch.Generator().manual_seed(2147483647)
C=torch.randn((27,2), generator=g)
w1 = torch.randn((6, 100), generator=g)
b1 = torch.randn((100), generator=g)
w2 = torch.randn((100, 27), generator=g)
b2 = torch.randn((27), generator=g)

In [202]:
parameters = [w1,b1,w2,b2]
sum(p.nelement() for p in parameters)

3427

In [203]:
for p in parameters:
    p.requires_grad= True

In [204]:
lre = torch.linspace(-3,0,1000) # learning rate exponent
lrs = 10**lre # learning rates


In [224]:
lri = []
lossi = []
for i in range(10000):

    # Minibatch construction
    ix = torch.randint(0, X.shape[0], (32,))

    # Forward pass
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1, 6) @ w1 + b1)
    logits = h @ w2 + b2
    loss = F.cross_entropy(logits, Y[ix])
    # print(loss.item())
    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    # lr = lrs[i]
    lr= 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # Track stats
    lri.append(lr)
    lossi.append(loss.item())



In [211]:
# plt.plot(lri, lossi)

In [225]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ w1 + b1)
logits = h @ w2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

2.4201431274414062